In [1]:
!pip install PyMySQL


In [2]:
!pip install spotipy

In [32]:
import pymysql.cursors

HOST = '127.0.0.1'
USERNAME = 'root'
PASSWORD = 'ENTER PASSWORD HERE'
DB = 'group20'

#HOST = 'ro2padgkirvcf55m.cbetxkdyhwsb.us-east-1.rds.amazonaws.com'
#USERNAME = 'kiobxvha0j25kdji'
#PASSWORD = 'ebrkilp5hgrfm8mg'
#DB = 'slztvi8pp04q6647'

connection = pymysql.connect(host= HOST,
                             user= USERNAME,
                             password= PASSWORD,
                             database= DB ,
                             cursorclass=pymysql.cursors.DictCursor)



In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = '4b82e53952b14228864022d7bea0e648'
secret = '01f7cf41fcca4c9eade8d50f2ab043a5'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [3]:
import json
with open('mpd.slice.0-999.json') as f:
    d = json.load(f)

In [4]:
import time

In [5]:
# This is query (returns all values (rows))
def query(statement, arguments = ()):
  with connection.cursor() as cursor:
    cursor.execute(statement, arguments)
    return cursor.fetchall()

In [6]:
# Insert record
def insert(statement, arguments = ()):
  with connection.cursor() as cursor:
    cursor.execute(statement, arguments)
    connection.commit()

In [23]:
# Show all the tables
SHOW_TABLES = "show tables;"
result = query(SHOW_TABLES)
result

[{'Tables_in_group20': 'album'},
 {'Tables_in_group20': 'album_artist'},
 {'Tables_in_group20': 'artist'},
 {'Tables_in_group20': 'artist_genre'},
 {'Tables_in_group20': 'audio_features'},
 {'Tables_in_group20': 'genre'},
 {'Tables_in_group20': 'playlist'},
 {'Tables_in_group20': 'playlist_tracks'},
 {'Tables_in_group20': 'track'},
 {'Tables_in_group20': 'track_artist'},
 {'Tables_in_group20': 'user'},
 {'Tables_in_group20': 'user_playlist'}]

In [59]:
# Insert the Spotify Account into the 'user' table
STATEMENT = 'insert into user (display_name, user_uri) values (%s, %s)'
insert(STATEMENT, ('spotify', 'spotify:user:spotify'))

In [60]:
query('select * from user;')

[{'user_id': 1, 'display_name': 'spotify', 'user_uri': 'spotify:user:spotify'}]

In [61]:
# dictionaries to keep track of what has already been added to database
PLAYLIST_DICT = {} # map pid to bool
ALBUM_DICT = {} # map album_uri to album_id
TRACK_DICT = {} # map track_uri to track_id
ARTIST_DICT = {} # map artist_uri to artist_id
GENRE_DICT = {} # map genre_name to genre_id

# counters
playlist_id = 1
track_id = 1

In [48]:
# Playlists made by Spotify
playlists = sp.user_playlists('spotify')

In [76]:
# LOADING USER PLAYLISTS
for playlist in d['playlists']:
  #time.sleep(3)
  ### PLAYLIST INFO  

  # dictionary for keeping track of duplicate tracks in a playlist
  TR_PL_DICT = {} # maps track_id to dummy value

  # pid
  pid = playlist['pid'] + 1
    
  if pid in PLAYLIST_DICT:
    continue

  PLAYLIST_DICT[pid] = 1

  # Playlist name
  name = playlist['name']
    
  # collaborative?
  if playlist['collaborative']:
    collaborative = 1
  else:
    collaborative = 0

  # num tracks
  num_tracks = playlist['num_tracks']
    
  # followers
  num_followers = playlist['num_followers']

  PLAYLIST_INSERT = "insert into playlist values (%s, %s, %s, %s, %s)"
  try:
    insert(PLAYLIST_INSERT, (pid, name, collaborative, num_tracks, num_followers))
  except:
    continue

  ### TRACK INFO
  for track in playlist['tracks']:
    
    # track uri
    uri = track['track_uri']
    
    API_TRACK = sp.track(uri) # fetch track info from api
    
    PL_TR_INSERT = "insert into playlist_tracks (pid, track_id) values (%s, %s)"
    
    if uri in TRACK_DICT: # if track is already in database
        if TRACK_DICT[uri] not in TR_PL_DICT:
            insert(PL_TR_INSERT, (pid, TRACK_DICT[uri]))
            TR_PL_DICT[TRACK_DICT[uri]] = 1
        continue
    
    # track name 
    track_name = track['track_name']
    
    # track popularity
    popularity = API_TRACK['popularity']
    
    # track duration
    duration = track['duration_ms']
    
    # array of all artist_ids for track
    TR_ARTISTS = []
    
    ### ARTISTS and TRACK ARTISTS
    for artist in API_TRACK['artists']:
        
        
        if artist['uri'] not in ARTIST_DICT:
            artist_uri = artist['uri']
            artist_name = artist['name']
        
            ARTIST_INSERT = 'insert into artist (artist_uri, artist_name) values (%s, %s)'
            insert(ARTIST_INSERT, (artist_uri, artist_name))
            time.sleep(0.01)
            
            query_artist_id = 'select artist_id from artist where artist_uri = %s;'
            artist_id = query(query_artist_id, (artist_uri))[0]['artist_id']
            ARTIST_DICT[artist_uri] = artist_id
            
            ### GENRE and ARTIST GENRES
            for genre in sp.artist(artist_uri)['genres']:
                
                if genre not in GENRE_DICT:
                    GENRE_INSERT = 'insert into genre (genre_name) values (%s)'
                    insert(GENRE_INSERT, (genre))
                    time.sleep(0.01)
                    
                    query_genre_id = 'select genre_id from genre where genre_name = %s'
                    genre_id = query(query_genre_id, (genre))[0]['genre_id']
                    GENRE_DICT[genre] = genre_id
                else:
                    genre_id = GENRE_DICT[genre]
                
                ART_G_INSERT = 'insert into artist_genre (artist_id, genre_id) values (%s, %s)'
                insert(ART_G_INSERT, (artist_id, genre_id))
            
        else:
            artist_id = ARTIST_DICT[artist['uri']]
        
        TR_ARTISTS.append(artist_id)
        
    ### ALBUM INFO
    if track['album_uri'] in ALBUM_DICT: # if album is already in database
        
        album_id = ALBUM_DICT[track['album_uri']]
        
    else:
    
        # album name
        album_name = track['album_name']
    
        # album uri
        album_uri = track['album_uri']
    
        # number of tracks
        num_tracks = sp.album(album_uri)['total_tracks']
    
        ALBUM_INSERT = 'insert into album (album_name, album_uri, num_tracks) values (%s, %s, %s)'
        insert(ALBUM_INSERT, (album_name, album_uri, num_tracks))
        time.sleep(0.01)
        
        query_album_id = 'select album_id from album where album_uri = %s;'
        album_id = query(query_album_id, (album_uri))[0]['album_id']
        ALBUM_DICT[album_uri] = album_id
        
        # ALBUM_ARTIST
        for artist in API_TRACK['artists']:

            if artist['uri'] in ARTIST_DICT:
                query_artist_id = 'select artist_id from artist where artist_uri = %s'
                alb_artist_id = query(query_artist_id, (artist['uri']))[0]['artist_id']
        
                ALB_ART_INSERT = 'insert into album_artist (album_id, artist_id) values (%s, %s)'
                insert(ALB_ART_INSERT, (album_id, alb_artist_id))
                
    
    
    
    try:
        TRACK_INSERT = 'insert into track (track_id, track_name, track_uri, album_id, duration, popularity) values (%s,%s,%s,%s,%s,%s)'
        insert(TRACK_INSERT, (track_id, track_name, uri, album_id, duration, popularity))
        
        TRACK_DICT[uri] = track_id
        insert(PL_TR_INSERT, (pid, track_id))
        TR_PL_DICT[track_id] = 1
    
        TR_ARTIST_INSERT = 'insert into track_artist (track_id, artist_id) values (%s, %s)'
        for artist_id in TR_ARTISTS:
                insert(TR_ARTIST_INSERT, (track_id, artist_id))
    
        ### AUDIO FEATURES
        features = sp.audio_features(uri)[0]
        if features is not None:
            danceability = features["danceability"]
            acousticness = features["acousticness"]
            energy = features["energy"]
            loudness = features["loudness"]
            liveness = features["liveness"]
    
            FEATURE_INSERT = 'insert into audio_features values (%s, %s, %s, %s, %s, %s)'
            insert(FEATURE_INSERT, (danceability, acousticness, energy, loudness, liveness, track_id))
    
        track_id += 1
    except:
        track_id += 1
        continue
    
  #playlist_id += 1

In [88]:
playlist_id = 1001

In [87]:
SP_PL_DICT = {}

In [89]:
# ITERATE THROUGH ALL OF SPOTIFY'S PLAYLISTS
for playlist in playlists['items']:
  time.sleep(3)

  ### PLAYLIST INFO  

  # Playlist name
  name = playlist['name']
    
  if name in SP_PL_DICT:
    continue

  SP_PL_DICT[name] = 1
    
  # collaborative?
  if playlist['collaborative']:
    collaborative = 1
  else:
    collaborative = 0

  # num tracks
  num_tracks = playlist['tracks']['total']

  PLAYLIST_INSERT = "insert into playlist (pid, name, collaborative, num_tracks) values (%s, %s, %s, %s)"
  insert(PLAYLIST_INSERT, (playlist_id, name, collaborative, num_tracks))
  USER_PL_INSERT = "insert into user_playlist (user_id, pid) values (%s, %s)"
  insert(USER_PL_INSERT, (1, playlist_id))
    
  ### TRACK INFO
  for track in sp.playlist_tracks(playlist['uri'])['items']:
        
    if track['track'] is None: # error handling
        continue
        
    # track uri
    uri = track['track']['uri']
    
    PL_TR_INSERT = "insert into playlist_tracks (pid, track_id) values (%s, %s)"
    
    if uri in TRACK_DICT: # if track is already in database
        insert(PL_TR_INSERT, (playlist_id, TRACK_DICT[uri]))
        continue
    
    # track name 
    track_name = track['track']['name']
    
    # track popularity
    popularity = track['track']['popularity']
    
    # track duration
    duration = track['track']['duration_ms']
    
    # array of all artist_ids for track
    TR_ARTISTS = []
    
    ### ARTISTS and TRACK ARTISTS
    for artist in track['track']['artists']:
        
        
        if artist['uri'] not in ARTIST_DICT:
            artist_uri = artist['uri']
            artist_name = artist['name']
        
            ARTIST_INSERT = 'insert into artist (artist_uri, artist_name) values (%s, %s)'
            insert(ARTIST_INSERT, (artist_uri, artist_name))
            time.sleep(0.01)
            
            query_artist_id = 'select artist_id from artist where artist_uri = %s;'
            artist_id = query(query_artist_id, (artist_uri))[0]['artist_id']
            ARTIST_DICT[artist_uri] = artist_id
            
            ### GENRE and ARTIST GENRES
            for genre in sp.artist(artist_uri)['genres']:
                
                if genre not in GENRE_DICT:
                    GENRE_INSERT = 'insert into genre (genre_name) values (%s)'
                    insert(GENRE_INSERT, (genre))
                    time.sleep(0.01)
                    
                    query_genre_id = 'select genre_id from genre where genre_name = %s'
                    genre_id = query(query_genre_id, (genre))[0]['genre_id']
                    GENRE_DICT[genre] = genre_id
                else:
                    genre_id = GENRE_DICT[genre]
                
                ART_G_INSERT = 'insert into artist_genre (artist_id, genre_id) values (%s, %s)'
                insert(ART_G_INSERT, (artist_id, genre_id))
            
        else:
            artist_id = ARTIST_DICT[artist['uri']]
            
        TR_ARTISTS.append(artist_id)
    
    ### ALBUM INFO
    if track['track']['album']['uri'] in ALBUM_DICT: # if album is already in database
        
        album_id = ALBUM_DICT[track['track']['album']['uri']]
        
    else:
    
        # album name
        album_name = track['track']['album']['name']
    
        # album uri
        album_uri = track['track']['album']['uri']
    
        # number of tracks
        num_tracks = track['track']['album']['total_tracks']
    
        ALBUM_INSERT = 'insert into album (album_name, album_uri, num_tracks) values (%s, %s, %s)'
        insert(ALBUM_INSERT, (album_name, album_uri, num_tracks))
        time.sleep(0.01)
        
        query_album_id = 'select album_id from album where album_uri = %s;'
        album_id = query(query_album_id, (album_uri))[0]['album_id']
        ALBUM_DICT[album_uri] = album_id
        
        # ALBUM_ARTIST
        for artist in track['track']['album']['artists']:

            if artist['uri'] in ARTIST_DICT:
                query_artist_id = 'select artist_id from artist where artist_uri = %s'
                alb_artist_id = query(query_artist_id, (artist['uri']))[0]['artist_id']
        
                ALB_ART_INSERT = 'insert into album_artist (album_id, artist_id) values (%s, %s)'
                insert(ALB_ART_INSERT, (album_id, alb_artist_id))
    
    try:
        TRACK_INSERT = 'insert into track (track_id, track_name, track_uri, album_id, duration, popularity) values (%s,%s,%s,%s,%s,%s)'
        insert(TRACK_INSERT, (track_id, track_name, uri, album_id, duration, popularity))
        TRACK_DICT[uri] = track_id
        insert(PL_TR_INSERT, (playlist_id, track_id))
    
        TR_ARTIST_INSERT = 'insert into track_artist (track_id, artist_id) values (%s, %s)'
        for artist_id in TR_ARTISTS:
            insert(TR_ARTIST_INSERT, (track_id, artist_id))
    
        ### AUDIO FEATURES
        features = sp.audio_features(uri)[0]
        if features is not None:
            danceability = features["danceability"]
            acousticness = features["acousticness"]
            energy = features["energy"]
            loudness = features["loudness"]
            liveness = features["liveness"]
    
            FEATURE_INSERT = 'insert into audio_features values (%s, %s, %s, %s, %s, %s)'
            insert(FEATURE_INSERT, (danceability, acousticness, energy, loudness, liveness, track_id))
    
        track_id += 1
    except:
        track_id += 1
        continue
    
  playlist_id += 1
    
    